## 1. Summarize Text with Hugging Face BART

In [1]:

from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = "Innovatek, the multinational technology conglomerate known for its innovations in artificial intelligence and consumer electronics, released its quarterly earnings report on Monday. The results exceeded Wall Street expectations, showing a robust 20% increase in revenue compared to the same quarter last year. This marks the fourth consecutive quarter of double-digit growth for the company.

Analysts attribute much of this surge to the strong sales performance of Innovatek's newly launched line of AI-powered consumer products, including the highly anticipated voice-activated home assistant “Neura” and the wearable health monitor “PulseSync.” These products have seen strong adoption rates across North America and parts of Asia, particularly in urban tech-savvy markets.

In a press briefing, CEO Maria Lin highlighted the company’s strategic focus on research and development, revealing that over 18% of its quarterly revenue was reinvested into future-facing technologies, including generative AI and neuromorphic computing. “We are committed to innovation not only as a product strategy but as a foundational value,” Lin said.

Looking ahead, Innovatek plans to expand aggressively into international markets, with special attention on Latin America and Eastern Europe. The company is also planning to double the size of its AI research division by next year and is in talks to open a new research hub in Helsinki, Finland. Market response to the news was largely positive, with Innovatek shares climbing 5.7% in after-hours trading.

"
summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
print(summary[0]['summary_text'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 130, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Long article text goes here. Long article text going here... Click here to read the rest of the article. Scroll down for the next section.


In [4]:
text = "Innovatek, the multinational technology conglomerate known for its innovations in artificial intelligence and consumer electronics, released its quarterly earnings report on Monday. The results exceeded Wall Street expectations, showing a robust 20% increase in revenue compared to the same quarter last year. This marks the fourth consecutive quarter of double-digit growth for the company. Analysts attribute much of this surge to the strong sales performance of Innovatek's newly launched line of AI-powered consumer products, including the highly anticipated voice-activated home assistant “Neura” and the wearable health monitor “PulseSync.” These products have seen strong adoption rates across North America and parts of Asia, particularly in urban tech-savvy markets."
summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

Innovatek reported a 20% increase in revenue compared to the same quarter last year. This marks the fourth consecutive quarter of double-digit growth for the company. Analysts attribute much of this surge to the company's newly launched line of AI-powered consumer products.


## 2. Fine-tune BART on CNN/DailyMail

In [ ]:

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def preprocess(examples):
    inputs = tokenizer(examples["article"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(examples["highlights"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

training_args = TrainingArguments(
    output_dir="./results", per_device_train_batch_size=4, num_train_epochs=1, logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


## 3. Extractive vs. Abstractive Summarization

In [ ]:

# Extractive with spaCy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from heapq import nlargest

text = "Long article text here..."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

word_freq = {}
for word in doc:
    if word.text.lower() not in STOP_WORDS and word.is_alpha:
        word_freq[word.text.lower()] = word_freq.get(word.text.lower(), 0) + 1

sentence_scores = {}
for sent in doc.sents:
    for word in sent:
        if word.text.lower() in word_freq:
            sentence_scores[sent] = sentence_scores.get(sent, 0) + word_freq[word.text.lower()]

summary_sentences = nlargest(3, sentence_scores, key=sentence_scores.get)
summary = " ".join([sent.text for sent in summary_sentences])
print(summary)


## 4. Summarize Long Documents

In [ ]:

def split_text(text, chunk_size=1024):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

chunks = list(split_text("Long document text", chunk_size=400))

from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary_parts = [summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text'] for chunk in chunks]
full_summary = " ".join(summary_parts)
print(full_summary)


## 5. ROUGE Evaluation

In [ ]:

from datasets import load_metric

rouge = load_metric("rouge")
predictions = ["Bart summarized this well."]
references = ["The summary should cover key points clearly."]
results = rouge.compute(predictions=predictions, references=references)
print(results)


## 6. Prompt-based Summarization (Chat Models)

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

prompt = "Summarize this article:\n" + "Long article..." + "\nSummary:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output_ids = model.generate(input_ids, max_new_tokens=150)
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(summary)


## 7. Batch Summarization from CSV

In [ ]:

import pandas as pd
from transformers import pipeline

df = pd.read_csv("articles.csv")  # column: content
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
df["summary"] = df["content"].apply(lambda x: summarizer(x, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])
df.to_csv("summaries.csv", index=False)


## 8. REST API with FastAPI

In [ ]:

# Save this as app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

class TextRequest(BaseModel):
    text: str

@app.post("/summarize")
def summarize(req: TextRequest):
    result = summarizer(req.text, max_length=130, min_length=30, do_sample=False)
    return {"summary": result[0]['summary_text']}


## 9. Quantized Summarization (4-bit LLM)

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)
model = AutoModelForCausalLM.from_pretrained("TheBloke/LLaMA-2-7B-GGML", quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("TheBloke/LLaMA-2-7B-GGML")


## 10. Multilingual Summarization (mBART)

In [ ]:

from transformers import MBartTokenizer, MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")

text = "Texte en français ici..."  # French text
tokenizer.src_lang = "fr_XX"

input_ids = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).input_ids
summary_ids = model.generate(input_ids, max_length=100)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)
